In [1]:

import datetime
from dateutil.relativedelta import relativedelta
import json
import pandas as pd
import psycopg2
import plotly.express as px

In [35]:
conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='',
    host='',  # e.g., 'localhost'
    port='5432'   # default is 5432
)

def fetch_time_query(time):
    if time == 'daily':
        date = '2024-10-10'
        query = f"""
            SELECT * 
            FROM era5
            WHERE DATE(time) = '{date.strftime('%Y-%m-%d')};'
        """
    elif time == 'monthly':
        year, month = 2024,10
        start_date = datetime.date(year,month,1)
        end_date = start_date + relativedelta(months=1)
        query = f"""
            SELECT *
			FROM era5_daily e, residential_consumption_aggregated r
			WHERE e.postal_code = r.postalcode and r."date" = e."date" and e."date"  >= '{start_date}' AND e."date"  < '{end_date}';
        """
    elif time == 'annual':
        year= 2023
        start_date = datetime.date(year,1,1)
        end_date = datetime.date(year+1,1,1)
        query = f"""
            SELECT *
            FROM era5_monthly e, residential_consumption_monthly r
            WHERE e.postal_code = r.postalcode and r.month = e.month and r.year=e.year and e.year='{year}';
        """
    return  query

province_mapping = {
    '0':'Barcelona',
    '1':'Girona',
    '2':'Lleida',
    '4':'Tarragona'
}

In [36]:
query = fetch_time_query("annual")
df_era5 = pd.read_sql_query(query, conn)
df_era5

/tmp/ipykernel_525145/411972724.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,month,postal_code,airtemperature,cdd,hdd,relativehumidity,windspeed,winddirection,ghi,dni,sunelevation,year,month,postalcode,consumption,contracts
0,2023,01,08001,-1.698195,0.000000,14655.457415,77.674707,1.606126,297.133632,90.448124,146.269243,-16.393550,2023,01,08001,5056073.0,19506
1,2023,01,08002,-2.162166,0.000000,15000.651824,77.917161,1.563474,300.117511,91.643447,153.086939,-16.394065,2023,01,08002,3824433.0,11035
2,2023,01,08003,-2.941902,0.000000,15580.775353,78.249421,1.534985,304.241027,93.775452,164.977482,-16.394420,2023,01,08003,6023888.0,22238
3,2023,01,08004,-0.753737,0.000000,13952.780601,77.164957,1.780361,288.627383,88.567227,135.524378,-16.392039,2023,01,08004,5419367.0,23987
4,2023,01,08005,-4.062783,0.000000,16414.710627,78.950001,1.469888,119.806644,95.418575,174.685151,-16.396946,2023,01,08005,5550615.0,16961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12629,2023,12,43893,5.116208,0.000000,9585.541133,71.595939,2.241787,265.981439,72.852877,108.657683,-18.123120,2023,12,43893,1004247.0,5282
12630,2023,12,43894,7.175505,0.000000,8053.424422,74.874145,1.479304,259.940346,81.000723,136.496153,-18.040203,2023,12,43894,593153.0,1944
12631,2023,12,43895,8.240424,1.064332,7262.189234,75.814283,1.845381,236.948027,80.239980,136.134894,-18.047558,2023,12,43895,981782.0,4988
12632,2023,12,43896,5.792395,0.000000,9082.458485,73.686534,1.262525,286.433702,81.809070,138.707727,-18.033198,2023,12,43896,710090.0,2401


In [75]:
df = df_era5[df_era5['postal_code'] == '08031']
df

,year,month,postal_code,airtemperature,cdd,hdd,relativehumidity,windspeed,winddirection,ghi,dni,sunelevation,year,month,postalcode,consumption,contracts
30,2023,01,08031,-2.870546,0.000000,15527.685934,75.153293,1.314287,178.138431,94.402501,165.068912,-16.403843,2023,01,08031,4561145.0,22123
1063,2023,02,08031,0.969735,0.000000,11444.338297,62.826475,0.911372,220.929112,143.643460,230.157816,-10.206858,2023,02,08031,4057045.0,22075
2102,2023,03,08031,11.309702,158.407258,5129.298337,63.934851,1.856369,234.011470,193.047854,227.933405,-1.363303,2023,03,08031,3218835.0,20059
3158,2023,04,08031,10.977544,94.209995,5150.377964,65.660669,1.092750,239.491522,236.290452,209.948645,7.861265,2023,04,08031,2627885.0,20052
4206,2023,05,08031,14.709253,460.472905,2908.788947,62.090648,1.287706,161.153273,260.076201,200.984366,15.009873,2023,05,08031,3329187.0,23832
5263,2023,06,08031,21.935979,3202.274334,368.369295,65.425246,1.358356,161.343804,268.804299,176.370526,18.253748,2023,06,08031,3382105.0,21942
6320,2023,07,08031,24.581639,4951.930705,55.191385,64.513119,1.761779,194.133594,291.062240,220.414442,16.764472,2023,07,08031,3767520.0,20085
7378,2023,08,08031,22.240853,3601.939577,446.745239,65.298405,1.089683,198.580608,251.696646,218.726478,10.878452,2023,08,08031,3639832.0,20905
8435,2023,09,08031,21.858803,3045.341878,267.003917,64.362784,1.947760,150.181170,207.304006,230.435673,2.261496,2023,09,08031,2860642.0,19538
9492,2023,10,08031,13.369253,446.170112,3896.076870,76.249625,1.153563,193.495638,137.705593,158.903194,-7.114987,2023,10,08031,2748045.0,18762


In [76]:
months = df['month'].iloc[:,0].values
temperature = df['airtemperature'].values
consumption = df['consumption'].values

In [56]:
consumption = [160632., 147501., 113761.,  92067.,  95051., 103468., 136871.,
       139943.,  99275.,  98302., 109159., 147070.]

In [74]:
min(temperature)-0.1*min(temperature)

np.float64(2.272420038025046)

In [77]:
import plotly.graph_objects as go
from plotly.data import tips

df = tips()

fig = go.Figure(
    data=go.Bar(
        x=months,
        y=temperature,
        name="Air temperature",
        marker=dict(color="paleturquoise"),
    )
)
fig.add_trace(
    go.Scatter(
        x=months,
        y=consumption,
        yaxis="y2",
        name="Consumption",
        marker=dict(color="crimson"),
    )
)

fig.update_layout(
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Air temperature"),
        side="left",
        range=[0,27.205693815532594],
    ),
    yaxis2=dict(
        title=dict(text="Consumption"),
        side="right",
        range=[82860.3, 176695.2],
        overlaying="y",
        tickmode="sync",
    ),
)

fig.show()